In [1]:
%pylab inline
import pandas as pd
import os 

Populating the interactive namespace from numpy and matplotlib


Construcción base de datos:

In [2]:
#Inicializo las columnas con los nombres de las localidades que ya tienen datos
localidades=os.listdir("Votaciones_localidades")
Nombreslocalidades=[]
for i in localidades:
    Nombreslocalidades.append(i[:-7])
df1 = pd.DataFrame(columns=[Nombreslocalidades])
df1= df1.rename_axis(None)

In [3]:
#Algoritmo que ingresa los datos a la tabla, la tabla solo contiene las localidades reportadas.
Index=('P_Liberal','P_Conservador','P_Opcion_Ciudadana','P_Cambio_Radical','P_Alianza_Verde','P_MIRA','P_de_la_U','Polo','P_Centro_Democratico','P_SOMOS','FARC','Colombia_Justa_Libres','SI_SE_PUEDE',"Union_con_Fortaleza","Todos_somos_Colombia",'Coalicion_Decencia','Voto_Blanco_Nacional','AICO','ASI','MAIS','SOBERANIA','MIA','RenovacionEtnica','CASIYOUREN','Voto_Blanco_Indigena','Nulos','No_Marcados',"Potencial votantes", "Referencia")
for i in localidades:
    temp=pd.read_csv("Votaciones_localidades/"+str(i))
    temp.index = Index
    df1[str(i[:-7])] = temp.iloc[:-1,1].apply(pd.to_numeric)

In [4]:
#Porcentaje voto valido con datos escrutados
a=df1.sum(axis=1)[:-11]
b=a.sum()
c=100*a/b
c

P_Liberal                 6.131043
P_Conservador             4.074415
P_Opcion_Ciudadana        1.129080
P_Cambio_Radical          7.809012
P_Alianza_Verde          22.525637
P_MIRA                    4.525689
P_de_la_U                 4.586081
Polo                      6.285812
P_Centro_Democratico     19.164078
P_SOMOS                   0.978314
FARC                      0.413881
Colombia_Justa_Libres     4.631268
SI_SE_PUEDE               0.085860
Union_con_Fortaleza       0.136413
Todos_somos_Colombia      0.383515
Coalicion_Decencia        9.052103
Voto_Blanco_Nacional      8.087800
dtype: float64

In [8]:
a

P_Liberal                143958
P_Conservador             95668
P_Opcion_Ciudadana        26511
P_Cambio_Radical         183357
P_Alianza_Verde          528906
P_MIRA                   106264
P_de_la_U                107682
Polo                     147592
P_Centro_Democratico     449976
P_SOMOS                   22971
FARC                       9718
Colombia_Justa_Libres    108743
SI_SE_PUEDE                2016
Union_con_Fortaleza        3203
Todos_somos_Colombia       9005
Coalicion_Decencia       212545
Voto_Blanco_Nacional     189903
dtype: int64